In [1]:
# Import necessary libraries
import pandas as pd
import pyodbc
from datetime import datetime
import os

# 1. Load generated data into DataFrames
print("Loading CSV files...")
customers_df = pd.read_csv('customers.csv')
meter_readings_df = pd.read_csv('meter_readings.csv', parse_dates=['Timestamp'])

# 2. View sample data
print("\nCustomer Data Sample:")
print(customers_df.head())
print("\nMeter Readings Sample:")
print(meter_readings_df.head())

# 3. Perform transformations
print("\nTransforming data...")

# Create TimeID (YYYYMMDDHH format)
meter_readings_df['TimeID'] = (
    meter_readings_df['Timestamp'].dt.year * 1000000 +
    meter_readings_df['Timestamp'].dt.month * 10000 +
    meter_readings_df['Timestamp'].dt.day * 100 +
    meter_readings_df['Timestamp'].dt.hour
).astype(int)

# Create DimTime DataFrame
dim_time = pd.DataFrame({
    'TimeID': meter_readings_df['TimeID'].unique(),
    'Date': pd.to_datetime(meter_readings_df['Timestamp'].dt.date),
    'Hour': meter_readings_df['Timestamp'].dt.hour,
    'DayOfWeek': meter_readings_df['Timestamp'].dt.day_name()
}).drop_duplicates()

# Clean Fact table data
fact_energy = meter_readings_df[['MeterID', 'TimeID', 'CustomerID', 'kWh']].copy()
fact_energy['Cost'] = fact_energy['kWh'] * 0.15  # Example rate

# 4. Save cleaned data to folder
cleaned_data_dir = 'cleaned_data'
os.makedirs(cleaned_data_dir, exist_ok=True)

dim_time.to_csv(f'{cleaned_data_dir}/DimTime.csv', index=False)
customers_df.to_csv(f'{cleaned_data_dir}/DimCustomer.csv', index=False)
fact_energy.to_csv(f'{cleaned_data_dir}/FactEnergyConsumption.csv', index=False)

print(f"\nCleaned data saved to {cleaned_data_dir} folder")

# 5. Create connection to SQL Server (Windows Authentication)
print("\nConnecting to SQL Server...")
server_name = 'YOUR_SERVER_NAME'  # Replace with your server name
database_name = 'EnergyMonitoringMVP'

conn_str = (
    f"Driver={{ODBC Driver 17 for SQL Server}};"
    f"Server={server_name};"
    f"Database={database_name};"
    "Trusted_Connection=yes;"
)

try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Connection successful!")
    
    # 6. Clear existing data
    print("\nClearing existing warehouse data...")
    cursor.execute("TRUNCATE TABLE FactEnergyConsumption")
    cursor.execute("DELETE FROM DimTime")
    cursor.execute("DELETE FROM DimCustomer")
    conn.commit()
    
    # 7. Load new data
    print("Loading DimCustomer...")
    for index, row in customers_df.iterrows():
        cursor.execute("""
            INSERT INTO DimCustomer (CustomerID, Name, Address, TariffPlan)
            VALUES (?, ?, ?, ?)
            """, row['CustomerID'], row['Name'], row['Address'], row['TariffPlan'])
    
    print("Loading DimTime...")
    for index, row in dim_time.iterrows():
        cursor.execute("""
            INSERT INTO DimTime (TimeID, Date, Hour, DayOfWeek)
            VALUES (?, ?, ?, ?)
            """, row['TimeID'], row['Date'], row['Hour'], row['DayOfWeek'])
    
    print("Loading FactEnergyConsumption...")
    for index, row in fact_energy.iterrows():
        cursor.execute("""
            INSERT INTO FactEnergyConsumption (MeterID, TimeID, CustomerID, kWh, Cost)
            VALUES (?, ?, ?, ?, ?)
            """, row['MeterID'], row['TimeID'], row['CustomerID'], row['kWh'], row['Cost'])
    
    conn.commit()
    print("\nData successfully loaded to data warehouse!")
    
except Exception as e:
    print(f"Error: {str(e)}")
finally:
    if 'conn' in locals():
        conn.close()
    print("ETL process completed.")